In [24]:
import os

In [25]:
import random

import cv2
from matplotlib import pyplot as plt
import albumentations as A

In [26]:
classes_txt = open("C:/Users/bench/PycharmProjects/AISS_CV/classes.txt", "r")
lines = classes_txt.read().splitlines()
labels_dict = {}


i = 0
for label in lines : 
    labels_dict[str(i)] = label
    i += 1
    
print(labels_dict)    


{'0': 'Mensa', '1': 'AKK', '2': 'Audimax', '3': 'Neue Bib', '4': 'Alte Bib', '5': 'Studierendenwerk', '6': 'Lernzentrum', '7': 'Mathebau', '8': 'Harber-Bosch-Reaktor', '9': 'Statue am Ehrenhof', '10': 'Heinrich-Hertz-Denkmal', '11': 'Kolben', '12': 'Wärmeflasche', '13': 'Gründerschmiede'}


In [27]:
mypath = "C:/Users/bench/PycharmProjects/AISS_CV/Training Dataset/"
mydict = {}
def read_txt_yolo():
    txt_list = []
    for file in os.listdir(mypath):
        if file.endswith(".txt"):
            txt_list.append(file)

    for txt_name in txt_list:
        txt_path = mypath + txt_name
#       print("Input:" + txt_path)
        txt_file = open(txt_path, "r")
        lines = txt_file.read().splitlines()
#       print(lines)
        key = txt_name[:-4]
        mydict[key] = []
        for idx, line in enumerate(lines):
            value = line.split()
            x = y = w = h = cls = None
            cls = value[0]
            cls = labels_dict.get(cls)
            x = float(value[1])
            y = float(value[2])
            w = float(value[3])
            h = float(value[4])
#           print("CLASS = " + str(cls)  + "|####### X = " + str(x) + " |### Y ##### = " + str(y) + " |#### W #####| " + str(w) + " |####### H ########" + str(h) )
            mydict[key].append([x,y,w,h,cls])

    
read_txt_yolo()

print(mydict.get("AKK_ASTA_11")[0][:4])

[0.482143, 0.323909, 0.247024, 0.202712]


In [28]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=5):
    """Visualizes a single bounding box on the image"""
    x, y, w, h = bbox
    dh, dw, _ = img.shape
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    
    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1

    cv2.rectangle(img, (l, t), (r, b), (0, 0, 255),  thickness = thickness
                 )

    fl = "AKK"
    
#     cv2.putText(
#         img,
#         text=fl,
#         org=(x, y - int(0.3 * 4)),
#         fontFace=cv2.FONT_HERSHEY_SIMPLEX,
#         fontScale=0.35, 
#         color=TEXT_COLOR, 
#         lineType=cv2.LINE_AA,
#     )
    print("XMIN Y MIN " ,  x,y)
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    
    
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
        
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)
#     cv2.imwrite(r"./result/AKKMODY.jpg",img)


In [29]:
j = 0
for i in mydict.get('AKK_ASTA_11'):
    print(j)
    j= j+1
  

0
1


In [37]:
classes_txt = open("C:/Users/bench/PycharmProjects/AISS_CV/classes.txt", "r")
lines = classes_txt.read().splitlines()
category_id_to_name = {}


i = 0
for label in lines : 
    category_id_to_name[(i)] = label
    i += 1
    
print(category_id_to_name) 

{0: 'Mensa', 1: 'AKK', 2: 'Audimax', 3: 'Neue Bib', 4: 'Alte Bib', 5: 'Studierendenwerk', 6: 'Lernzentrum', 7: 'Mathebau', 8: 'Harber-Bosch-Reaktor', 9: 'Statue am Ehrenhof', 10: 'Heinrich-Hertz-Denkmal', 11: 'Kolben', 12: 'Wärmeflasche', 13: 'Gründerschmiede'}


In [38]:
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.7),
    A.Blur(blur_limit=7, always_apply=False, p=0.5),
], bbox_params=A.BboxParams(format='yolo',label_fields=["category_ids"]))

In [60]:
for file in os.listdir(mypath):
        if file.endswith(".jpg"):
            category_ids = 1
            print(mypath+file)
            random.seed(7)
            image = cv2.imread(mypath + file)
            img_name = file.rstrip(".jpg")
            bboxes = [mydict.get(str(img_name))][0:-1]
            print(bboxes)
            transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
            print(transformed['bboxes'])
            visualize(transformed['image'],transformed['bboxes'][0:-1],transformed['category_ids'],category_id_to_name)
            str1 = "".join(str(e) for e in transformed['bboxes'])
            cv2.imwrite("./results/"+ "simply_modified_"+img_name + ".jpg", transformed['image'], [int(cv2.IMWRITE_JPEG_QUALITY), 100])
            txt_file =open("./results/" + "simply_modified_"+img_name + ".txt", 'w')
            txt_file.write(str1)
            txt_file.close()
            print("saved new "+ img_name) 

C:/Users/bench/PycharmProjects/AISS_CV/Training Dataset/AKK_ASTA_1.jpg
[]
[]


TypeError: 'int' object is not iterable

In [53]:
txt_file =open("../data/obj/" + "imges_txt_train.txt", 'w')
for file in os.listdir(mypath):
    if file.endswith(".jpg"):
        img_name = file.rstrip(".jpg")
        str1 = ""+ "data/obj/" + img_name + ".jpg"
        txt_file.write(str1+ '\n')
        
txt_file.close()            